In [1]:
# Importing Libraries
import pandas as pd
import selenium                                  
from bs4 import BeautifulSoup
from selenium import webdriver                   
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import warnings                                  
warnings.filterwarnings("ignore")

import time                                      
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

#Importing required exceptions which needs to be handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# Let's Install automated chrome browser and connect it to web driver
driver = webdriver.Chrome(service=Service())

#maximise the window
driver.maximize_window()

In [2]:
#opening the website page
driver.get("https://www.nike.com")

In [3]:
# Clicking on shoes
allshoes = driver.find_elements(By.XPATH, '/html/body/div[3]/div/div[3]/header/div[1]/div[2]/nav/div[2]/div/ul/li[2]')

# Iterate through each element and click
for shoe in allshoes:
    shoe.click()

In [4]:
# Click on menshoes
menshoes = driver.find_elements(By.XPATH,'/html/body/div[4]/div/div/div[2]/div[2]/div/div/div/div/nav/div[1]/ul/li[1]/a')

# Iterate through each element and click
for shoe in menshoes:
    shoe.click()

In [5]:
driver.execute_script("window.scrollTo(0, 500);")

In [6]:
women_button=driver.find_element(By.XPATH,'//button[@aria-label="Filter for Women"]')
women_button.click()

In [11]:
while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(10)
    
    # Get the new height of the page
    new_height = driver.execute_script("return document.body.scrollHeight")

    # Break the loop if you have reached the bottom of the page
    if new_height == last_height:
        break

    # Update the last height
    last_height = new_height

In [16]:
product_url = []
name = []
category = []
colors = []

# Find all product URLs and add them to the list
urls = driver.find_elements(By.XPATH, '//a[@data-testid="product-card__link-overlay"]')
for url in urls:
    product_url.append(url.get_attribute('href'))

# Find_Product Card
card = driver.find_elements(By.XPATH, '//div[@class="product-card__body"]')

for i in card:
    nam = i.find_element(By.XPATH, './/div[@class="product-card__titles"]/div[1]')
    name.append(nam.text)

    cate = i.find_element(By.XPATH, './/div[@class="product-card__titles"]/div[2]')
    category.append(cate.text)

    try:
        col = i.find_element(By.XPATH, './/div[@class="product-card__product-count"]')
        colors.append(col.text.split(' ')[0])
    except NoSuchElementException:
        colors.append('NA')

In [17]:
len(product_url), len(name), len(category), len(colors)

(990, 990, 990, 990)

In [20]:
price = []
available_sizes = []
total_sizes_list = []
color_name = []
product_code=[]
review=[]
ratings=[]
features=[]
comfort=[]

for product in product_url:
    driver.get(product)
    time.sleep(1)
    
    #Extracting Price===========================================================================================
    try:
        price_element = driver.find_element(By.XPATH, '//div[@class="prl0-sm mb8-sm"]/div[1]/div/div[2]/div/div/div[1]/div')
        
        #extracting discounted Price
        try:
            reduced = price_element.find_element(By.XPATH, './/div[@data-test="product-price-reduced"]')
            price.append(reduced.text.split('₹')[1].strip())
        
        except NoSuchElementException:
            try:
                cost = price_element.find_element(By.XPATH, './/div[@data-test="product-price"]')
                price.append(cost.text.split('₹')[1].strip())
                
            except NoSuchElementException:
                price.append('NA')
                
    except NoSuchElementException:
        price.append('NA')
        
    
    # Extract Sizes and count _of_sizes==================================================================
    try:
        #Count of sizes
        size_table= driver.find_element(By.XPATH, '//div[@class="mt2-sm css-12whm6j"]')
        size_select = size_table.find_elements(By.XPATH, '//input[@name="skuAndSize" and not(@disabled)]')
        total_sizes_list.append(len(size_select))

        #Extracting Available sizes
        size_text=[size.get_attribute("value").split(':')[1] for size in size_select]
        available_sizes.append(size_text)
            
    except NoSuchElementException:
        available_sizes.append('NA')
        total_sizes_list.append('NA')
        
    # Extract name of the colors==========================================================================
    try:
        colr = driver.find_element(By.XPATH,'//ul[@class="description-preview__features pt8-sm pb6-sm ncss-list-ul"]/li[1]')
        color_name.append(colr.text.split(':')[1].strip())
        
    except NoSuchElementException:
        color_name.append('NA')
        
    # Extract Product code========================================================================================
    try:
        code = driver.find_element(By.XPATH,'//ul[@class="description-preview__features pt8-sm pb6-sm ncss-list-ul"]/li[2]')
        product_code.append(code.text.split(':')[1].strip())
    
    except NoSuchElementException:
        product_code.append('NA')
        
    #Extracting Review()================================================================================
    try:
        rev = driver.find_element(By.XPATH,'//details[@data-test="reviewsAccordionClick"]/summary/h3/span')
        review.append(rev.text.split('(')[1].strip(')'))
    
    except NoSuchElementException:
        review.append('NA')
        
        #extracting Star rating==================================================================================
    try:
        scroll = driver.find_element(By.XPATH,'//div[@class="css-w11ebw"]/details[2]')
        actions = ActionChains(driver)
        actions.move_to_element(scroll).perform()
        scroll.click()
        
    except NoSuchElementException:
        pass
    
    try:
        rat = driver.find_element(By.XPATH,'//div[@class="product-review mb10-sm"]/p[1]')
        ratings.append(rat.text.split(' ')[0])
        
    except NoSuchElementException:
        ratings.append('NA')
    
    try:
        button = driver.find_element(By.XPATH,'//button[@class="ncss-cta-primary-dark btn-lg underline prl0-sm pb0-sm readMoreBtn css-6q6seo"]')
        button.click()
        time.sleep(2)
    except NoSuchElementException:
        pass
    
    #extracting Features=====================================================================================
    try:
        popup= driver.find_element(By.XPATH,'//div[@class="css-1hwqoaj browse-dialog__panel browse-dialog__panel--drop-fade  css-1qiovem"]')
        
        descp =popup.find_element(By.XPATH,'//div[@class="pi-pdpmainbody"]')
        para = descp.find_elements(By.XPATH,'/p')
        features.append(descp.text)
        
    except NoSuchElementException:
        features.append('NA')
    
    #Extracting Comfort=========================================================================
    try:
        comf = popup.find_element(By.XPATH,'//div[@class="pi-pdpmainbody"]/ul[1]')
        comfort.append(comf.text)
    
    except (NoSuchElementException, StaleElementReferenceException):
        comfort.append('NA')

In [21]:
len(price), len(available_sizes), len(total_sizes_list), len(color_name), len(product_code), len(review), len(ratings), len(features), len(comfort)

(990, 990, 990, 990, 990, 990, 990, 990, 990)

In [22]:
# Converting in DataFrame
main_Nike_urls=pd.DataFrame({
    'Name': name,
    'Category': category,
    'Colors': colors,
    'Price': price,
    'Sizes': available_sizes,
    'Count_Sizes': total_sizes_list,
    'Color_Name': color_name,
    'product_Code': product_code,
    'Review': review,
    'Rating' : ratings,
    'Features' : features,
    'Comfort' : comfort,
    'Product_url': product_url
    
})

main_Nike_urls

,Name,Category,Colors,Price,Sizes,Count_Sizes,Color_Name,product_Code,Review,Rating,Features,Comfort,Product_url
0,Nike Air Force 1 '07,Men's Shoes,2,7 495.00,"[7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 1...",13,White/White,CW2288-111,1311,,LEGENDARY STYLE REFINED.\n\nThe radiance lives...,The stitched overlays on the upper add heritag...,https://www.nike.com/in/t/air-force-1-07-shoes...
1,Nike Air Max 1,Men's shoes,5,12 795.00,"[6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11,...",17,White/Photon Dust/Black/University Blue,FD9082-103,88,4.8,\nMeet the leader of the pack. Walking on clo...,"Plush and comfortable, the Max Air cushioning ...",https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
2,Nike Air Max 90,Men's Shoes,1,11 895.00,"[7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 1...",13,Anthracite/Black/Mystic Red/Summit White,FB9658-001,3,4.7,\nLace up and feel the legacy in this champio...,The textile upper with leather and synthetic o...,https://www.nike.com/in/t/air-max-90-shoes-0MB...
3,Jordan Max Aura 5,Men's Shoes,5,11 895.00,"[7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 1...",15,White/Varsity Red/Wolf Grey/Black,DZ4353-101,22,,"\nWhenyou need a shoe that's ready 24/7, it's...",Nike Air-Sole units provide lightweight cushio...,https://www.nike.com/in/t/jordan-max-aura-5-sh...
4,Nike Air Force 1 '07 Pro Tech,Men's Shoes,3,13 995.00,"[6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11,...",17,Light Silver/Clear/Light Silver,FB8875-002,8,4.6,\nThe radiance lives on in the Air Force 1 '0...,The leather upper with stitched overlays ages ...,https://www.nike.com/in/t/air-force-1-07-pro-t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,Nike Gripknit Phantom GX Elite Dynamic Fit FG ...,Custom Football Boot,2,NA,NA,NA,Multi-Colour/Multi-Colour,DX5266-902,NA,NA,NA,NA,https://www.nike.com/in/u/custom-nike-phantom-...
986,Nike Phantom GX Academy Dynamic Fit MG By You,Custom Multi-Ground Football Boot,5,NA,NA,NA,Multi-Colour/Multi-Colour/Multi-Colour,DX5260-900,NA,NA,NA,NA,https://www.nike.com/in/u/custom-phantom-acade...
987,Nike Mercurial Vapor 15 Academy By You,Custom Multi-Ground Football Boot,6,NA,NA,NA,Multi-Colour/Multi-Colour,FN6753-900,NA,NA,NA,NA,https://www.nike.com/in/u/custom-mercurial-vap...
988,Nike Phantom Luna FG By You,Custom Women's Firm-Ground Football Boot,6,NA,NA,NA,Multi-Colour/Multi-Colour,FJ4831-900,NA,NA,NA,NA,https://www.nike.com/in/u/custom-phantom-luna-...


In [23]:
# Saving in Csv

main_Nike_urls.to_csv('main_Nike_urls.csv', index=False)

In [24]:
color_links = []

for product in product_url:
    driver.get(product)

    try:
        color_container = driver.find_element(By.XPATH, '//fieldset[@class="colorway-images ta-sm-c d-lg-t"]')
        urls_colr = color_container.find_elements(By.XPATH, '//div[@class="css-7aigzk colorway-container d-sm-ib d-lg-tc pr1-sm pb1-sm"]')
        
        for i in urls_colr:
            i.click()
            current_url = driver.current_url
            
            # Check if the current URL is the same as the product URL
            if current_url != product:
                color_links.append(current_url)
                
    except NoSuchElementException:
        pass

In [25]:
len(color_links)

1606

In [27]:
colr_nike=pd.DataFrame({'color_links': color_links})
colr_nike

,color_links
0,https://www.nike.com/in/t/air-force-1-07-shoes...
1,https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
2,https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
3,https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
4,https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
...,...
1601,https://www.nike.com/in/t/phantom-gx-club-mult...
1602,https://www.nike.com/in/t/tiempo-legend-10-clu...
1603,https://www.nike.com/in/t/jordan-why-not-6-pf-...
1604,https://www.nike.com/in/t/jordan-why-not-6-pf-...


In [28]:
colr_nike.to_csv('colr_nike.csv', index = False)

In [30]:
color_links

['https://www.nike.com/in/t/air-force-1-07-shoes-WrLlWX/CW2288-001',
 'https://www.nike.com/in/t/air-max-1-shoes-ZCSX34/FD9082-104',
 'https://www.nike.com/in/t/air-max-1-shoes-ZCSX34/FD9082-102',
 'https://www.nike.com/in/t/air-max-1-shoes-ZCSX34/FD9082-100',
 'https://www.nike.com/in/t/air-max-1-shoes-ZCSX34/FD9082-101',
 'https://www.nike.com/in/t/jordan-max-aura-5-shoes-ZBZ4Pz/DZ4353-001',
 'https://www.nike.com/in/t/jordan-max-aura-5-shoes-ZBZ4Pz/DZ4353-061',
 'https://www.nike.com/in/t/jordan-max-aura-5-shoes-ZBZ4Pz/DZ4353-160',
 'https://www.nike.com/in/t/jordan-max-aura-5-shoes-ZBZ4Pz/DZ4353-003',
 'https://www.nike.com/in/t/jordan-max-aura-5-shoes-ZBZ4Pz/DZ4353-007',
 'https://www.nike.com/in/t/air-force-1-07-pro-tech-shoes-KHLmTh/FB8875-001',
 'https://www.nike.com/in/t/air-force-1-07-pro-tech-shoes-KHLmTh/FB8875-700',
 'https://www.nike.com/in/t/air-force-1-07-shoe-HWWX9W/CT2302-002',
 'https://www.nike.com/in/t/blazer-low-77-jumbo-shoes-xRGRJD/DR9865-400',
 'https://www.nik

In [57]:
links = color_links[:500]

In [58]:
links

['https://www.nike.com/in/t/air-force-1-07-shoes-WrLlWX/CW2288-001',
 'https://www.nike.com/in/t/air-max-1-shoes-ZCSX34/FD9082-104',
 'https://www.nike.com/in/t/air-max-1-shoes-ZCSX34/FD9082-102',
 'https://www.nike.com/in/t/air-max-1-shoes-ZCSX34/FD9082-100',
 'https://www.nike.com/in/t/air-max-1-shoes-ZCSX34/FD9082-101',
 'https://www.nike.com/in/t/jordan-max-aura-5-shoes-ZBZ4Pz/DZ4353-001',
 'https://www.nike.com/in/t/jordan-max-aura-5-shoes-ZBZ4Pz/DZ4353-061',
 'https://www.nike.com/in/t/jordan-max-aura-5-shoes-ZBZ4Pz/DZ4353-160',
 'https://www.nike.com/in/t/jordan-max-aura-5-shoes-ZBZ4Pz/DZ4353-003',
 'https://www.nike.com/in/t/jordan-max-aura-5-shoes-ZBZ4Pz/DZ4353-007',
 'https://www.nike.com/in/t/air-force-1-07-pro-tech-shoes-KHLmTh/FB8875-001',
 'https://www.nike.com/in/t/air-force-1-07-pro-tech-shoes-KHLmTh/FB8875-700',
 'https://www.nike.com/in/t/air-force-1-07-shoe-HWWX9W/CT2302-002',
 'https://www.nike.com/in/t/blazer-low-77-jumbo-shoes-xRGRJD/DR9865-400',
 'https://www.nik

In [123]:
name1 = []
category1=[]
colors1 = []
price1=[]
available_sizes1 = []
total_sizes_list1 = []
color_name1 = []
product_code1 =[]
review1 =[]
ratings1 =[]
features1 =[]
comfort1 =[]


for i in links:
    driver.get(i)
    
    #Extracting name
    try:
        nam =driver.find_element(By.XPATH,'//div[@class="prl0-sm mb8-sm"]/div/div/div/div/h1')
        name1.append(nam.text)
        
    except NoSuchElementException:
        name1.append('NA')
    
    # Extracting colors======================================================================================
    try:
        colr = driver.find_element(By.XPATH,'//fieldset[@class="colorway-images ta-sm-c d-lg-t"]')
        count= len(colr.find_elements(By.XPATH,'//div[@class="css-7aigzk colorway-container d-sm-ib d-lg-tc pr1-sm pb1-sm"]'))
        colors1.append(count)
        
    except NoSuchElementException:
        colors1.append('NA')
        
    # Extracting Price==========================================================================================
    try:
        price_element = driver.find_element(By.XPATH, '//div[@class="prl0-sm mb8-sm"]/div[1]/div/div[2]/div/div/div[1]/div')
        
        #extracting discounted Price
        try:
            reduced = price_element.find_element(By.XPATH, './/div[@data-test="product-price-reduced"]')
            price1.append(reduced.text.split('₹')[1].strip())
        
        except NoSuchElementException:
            try:
                cost = price_element.find_element(By.XPATH, './/div[@data-test="product-price"]')
                price1.append(cost.text.split('₹')[1].strip())
                
            except NoSuchElementException:
                price1.append('NA')
                
    except NoSuchElementException:
        price1.append('NA')
        
    #Extract Sizes and count _of_sizes==================================================================
    try:
        #Count of sizes
        size_table= driver.find_element(By.XPATH, '//div[@class="mt2-sm css-12whm6j"]')
        size_select = size_table.find_elements(By.XPATH, '//input[@name="skuAndSize" and not(@disabled)]')
        total_sizes_list1.append(len(size_select))

        #Extracting Available sizes
        size_text=[size.get_attribute("value").split(':')[1] for size in size_select]
        available_sizes1.append(size_text)
            
    except NoSuchElementException:
        available_sizes1.append('NA')
        total_sizes_list1.append('NA')
        
    # Extract name of the colors==========================================================================
    try:
        colr = driver.find_element(By.XPATH,'//ul[@class="description-preview__features pt8-sm pb6-sm ncss-list-ul"]/li[1]')
        color_name1.append(colr.text.split(':')[1].strip())
        
    except NoSuchElementException:
        color_name1.append('NA')
        
    # Extract Product code========================================================================================
    try:
        code = driver.find_element(By.XPATH,'//ul[@class="description-preview__features pt8-sm pb6-sm ncss-list-ul"]/li[2]')
        product_code1.append(code.text.split(':')[1].strip())
    
    except NoSuchElementException:
        product_code1.append('NA')
        
    #Extracting Review()================================================================================
    try:
        rev = driver.find_element(By.XPATH,'//details[@data-test="reviewsAccordionClick"]/summary/h3/span')
        review1.append(rev.text.split('(')[1].strip(')'))
    
    except NoSuchElementException:
        review1.append('NA')
        
    #extracting Star rating==================================================================================
    try:
        scroll = driver.find_element(By.XPATH,'//details[@data-test="reviewsAccordionClick"]')
        actions = ActionChains(driver)
        actions.move_to_element(scroll).perform()
        scroll.click()
        
    except NoSuchElementException:
        pass
    
    time.sleep(2)
    
    try:
        star_rat = driver.find_element(By.XPATH,'//div[@class="product-review mb10-sm"]')
        
        try:
            rat1 =star_rat.find_element(By.XPATH,'./p[1]')
            ratings1.append(rat1.text.split(' ')[0])
        
        except NoSuchElementException:
            try:
                rat = driver.find_element(By.XPATH,'//div[@class="reviews-component mb5-sm"]/div/p[1]')
                ratings1.append(rat.text.split(' ')[0])
        
            except NoSuchElementException:
                ratings1.append('NA')
                
    except NoSuchElementException:
        ratings1.append('NA')
    
    #Clicking button to extract Product Details======
    
    driver.execute_script("window.scrollBy(0, -1000);")
    
    try:
        button = driver.find_element(By.XPATH,'//button[@class="ncss-cta-primary-dark btn-lg underline prl0-sm pb0-sm readMoreBtn css-6q6seo"]')
        button.click()
        time.sleep(2)
    except NoSuchElementException:
        pass
    
    #extracting Features===========================================================
    try:
        popup= driver.find_element(By.XPATH,'//div[@class="css-1hwqoaj browse-dialog__panel browse-dialog__panel--drop-fade  css-1qiovem"]')
        
        descp =popup.find_element(By.XPATH,'//div[@class="pi-pdpmainbody"]')
        para = descp.find_elements(By.XPATH,'/p')
        features1.append(descp.text)
        
    except NoSuchElementException:
        features1.append('NA')
        
        
    #Extracting Comfort====================================================================
    try:
        comf =popup.find_element(By.XPATH,'//div[@class="pi-pdpmainbody"]/ul[1]')
        comfort1.append(comf.text)
    
    except (NoSuchElementException, StaleElementReferenceException):
        comfort1.append('NA')

In [60]:
category2 = []

for i in links:
    driver.get(i)
    
    #Extracting Category============================================================================
    try:
        cate= driver.find_element(By.XPATH,'//div[@class="prl0-sm mb8-sm"]/div/div/div/div/h2')
        category2.append(cate.text)
        
    except NoSuchElementException:
        category2.append('NA') 

In [61]:
len(category2)

500

In [124]:
len(colors1), len(price1), len(color_name1), len(product_code1), len(total_sizes_list1), len(available_sizes1), len(review1), len(ratings1)

(500, 500, 500, 500, 500, 500, 500, 500)

In [62]:
cat=pd.DataFrame({'Category':category2})
cat

,Category
0,Men's Shoes
1,Men's shoes
2,Men's shoes
3,Men's shoes
4,Men's shoes
...,...
495,Women's Road Running Shoes
496,Women's Road Running Shoes
497,Women's Road Running Shoes
498,Men's Hard Court Tennis Shoes


In [64]:
Data=pd.read_csv('Nike_clrurls1st_500.csv')
Data

,Name,Colors,Price,Sizes,Count_Sizes,Color_Name,product_Code,Review,Rating,Features,Comfort,Product_url
0,Nike Air Force 1 '07,2,7 495.00,"['6', '6.5', '7', '7.5', '8', '8.5', '9', '9.5...",17.0,Black/Black,CW2288-001,1311,4.8,LEGENDARY STYLE REFINED.\n\nThe radiance lives...,The stitched overlays on the upper add heritag...,https://www.nike.com/in/t/air-force-1-07-shoes...
1,Nike Air Max 1,5,12 795.00,"['6', '6.5', '7', '7.5', '8', '8.5', '9', '9.5...",16.0,White/Black/Pure Platinum/University Gold,FD9082-104,89,4.8,\nMeet the leader of the pack. Walking on clo...,"Plush and comfortable, the Max Air cushioning ...",https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
2,Nike Air Max 1,5,12 795.00,"['6', '6.5', '7', '7.5', '8', '8.5', '9', '9.5...",16.0,White/Pure Platinum/Medium Olive/Black,FD9082-102,89,4.8,\nMeet the leader of the pack. Walking on clo...,"Plush and comfortable, the Max Air cushioning ...",https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
3,Nike Air Max 1,5,12 795.00,"['7', '7.5', '8', '8.5', '9', '9.5', '10', '10...",13.0,White/Deep Royal Blue/Pure Platinum/Black,FD9082-100,89,4.8,\nMeet the leader of the pack. Walking on clo...,"Plush and comfortable, the Max Air cushioning ...",https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
4,Nike Air Max 1,5,12 795.00,"['7', '7.5']",2.0,White/Pure Platinum/Black/University Red,FD9082-101,89,4.8,\nMeet the leader of the pack. Walking on clo...,"Plush and comfortable, the Max Air cushioning ...",https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
...,...,...,...,...,...,...,...,...,...,...,...,...
495,Nike Pegasus 40,11,11 895.00,"['6', '6.5', '7', '7.5', '8', '8.5', '9', '9.5...",10.0,White/Pure Platinum/Metallic Silver,DV3854-101,248,4.6,"\nA springy ride for any run, the Peg's famil...",The heel area optimises the landing space and ...,https://www.nike.com/in/t/pegasus-40-road-runn...
496,Nike Pegasus 40,11,11 895.00,"['6', '6.5', '7', '7.5', '8', '8.5', '9', '9.5...",10.0,Black/Iron Grey/White,DV3854-001,248,4.6,"\nA springy ride for any run, the Peg's famil...",The heel area optimises the landing space and ...,https://www.nike.com/in/t/pegasus-40-road-runn...
497,Nike Pegasus 40,11,11 297.00,NaN,NaN,White/Blue Tint/Laser Orange/Picante Red,DV3854-102,248,4.6,"\nA springy ride for any run, the Peg's famil...",The heel area optimises the landing space and ...,https://www.nike.com/in/t/pegasus-40-road-runn...
498,NikeCourt Air Zoom Vapor 11,3,14 247.00,"['7.5', '8', '9.5']",3.0,White/Kelly Green,DR6966-102,9,4.6,"\nValidated by the greats of the game, the Va...",The herringbone outsole uses a data-informed d...,https://www.nike.com/in/t/nikecourt-air-zoom-v...


In [131]:
# Converting in DataFrame
Nike_clrurls1st_500=pd.DataFrame({
    'Name': name1,
    'Colors': colors1,
    'Price': price1,
    'Sizes': available_sizes1,
    'Count_Sizes': total_sizes_list1,
    'Color_Name': color_name1,
    'product_Code': product_code1,
    'Review': review1,
    'Rating' : ratings1,
    'Features' : features1,
    'Comfort' : comfort1,
    'Product_url': links
    
})

main_Nike_urls

,Name,Category,Colors,Price,Sizes,Count_Sizes,Color_Name,product_Code,Review,Rating,Features,Comfort,Product_url
0,Nike Air Force 1 '07,Men's Shoes,2,7 495.00,"[7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 1...",13,White/White,CW2288-111,1311,,LEGENDARY STYLE REFINED.\n\nThe radiance lives...,The stitched overlays on the upper add heritag...,https://www.nike.com/in/t/air-force-1-07-shoes...
1,Nike Air Max 1,Men's shoes,5,12 795.00,"[6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11,...",17,White/Photon Dust/Black/University Blue,FD9082-103,88,4.8,\nMeet the leader of the pack. Walking on clo...,"Plush and comfortable, the Max Air cushioning ...",https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
2,Nike Air Max 90,Men's Shoes,1,11 895.00,"[7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 1...",13,Anthracite/Black/Mystic Red/Summit White,FB9658-001,3,4.7,\nLace up and feel the legacy in this champio...,The textile upper with leather and synthetic o...,https://www.nike.com/in/t/air-max-90-shoes-0MB...
3,Jordan Max Aura 5,Men's Shoes,5,11 895.00,"[7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 1...",15,White/Varsity Red/Wolf Grey/Black,DZ4353-101,22,,"\nWhenyou need a shoe that's ready 24/7, it's...",Nike Air-Sole units provide lightweight cushio...,https://www.nike.com/in/t/jordan-max-aura-5-sh...
4,Nike Air Force 1 '07 Pro Tech,Men's Shoes,3,13 995.00,"[6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11,...",17,Light Silver/Clear/Light Silver,FB8875-002,8,4.6,\nThe radiance lives on in the Air Force 1 '0...,The leather upper with stitched overlays ages ...,https://www.nike.com/in/t/air-force-1-07-pro-t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,Nike Gripknit Phantom GX Elite Dynamic Fit FG ...,Custom Football Boot,2,NA,NA,NA,Multi-Colour/Multi-Colour,DX5266-902,NA,NA,NA,NA,https://www.nike.com/in/u/custom-nike-phantom-...
986,Nike Phantom GX Academy Dynamic Fit MG By You,Custom Multi-Ground Football Boot,5,NA,NA,NA,Multi-Colour/Multi-Colour/Multi-Colour,DX5260-900,NA,NA,NA,NA,https://www.nike.com/in/u/custom-phantom-acade...
987,Nike Mercurial Vapor 15 Academy By You,Custom Multi-Ground Football Boot,6,NA,NA,NA,Multi-Colour/Multi-Colour,FN6753-900,NA,NA,NA,NA,https://www.nike.com/in/u/custom-mercurial-vap...
988,Nike Phantom Luna FG By You,Custom Women's Firm-Ground Football Boot,6,NA,NA,NA,Multi-Colour/Multi-Colour,FJ4831-900,NA,NA,NA,NA,https://www.nike.com/in/u/custom-phantom-luna-...


In [66]:
Data['Category']= cat['Category']

In [67]:
Data.shape

(500, 13)

In [68]:
# Saving in Csv

Data.to_csv('Nike_clrurls1st_500.csv', index=False)

In [15]:
links1 = color_links[500:1000]

In [16]:
links1

['https://www.nike.com/in/t/brsb-skate-shoes-RJF3Ss/DH9227-001',
 'https://www.nike.com/in/t/tech-hera-shoes-JlV5km/DR9761-101',
 'https://www.nike.com/in/t/tech-hera-shoes-JlV5km/DR9761-100',
 'https://www.nike.com/in/t/tech-hera-shoes-JlV5km/DR9761-110',
 'https://www.nike.com/in/t/tech-hera-shoes-JlV5km/DR9761-600',
 'https://www.nike.com/in/t/tech-hera-shoes-JlV5km/DR9761-002',
 'https://www.nike.com/in/t/air-vapormax-2023-flyknit-shoes-4LVgjW/DV6840-101',
 'https://www.nike.com/in/t/air-vapormax-2023-flyknit-shoes-4LVgjW/DV6840-002',
 'https://www.nike.com/in/t/air-vapormax-2023-flyknit-shoes-4LVgjW/DV6840-100',
 'https://www.nike.com/in/t/free-metcon-5-workout-shoes-HfHgmZ/DV3950-002',
 'https://www.nike.com/in/t/free-metcon-5-workout-shoes-HfHgmZ/DV3950-101',
 'https://www.nike.com/in/t/free-metcon-5-workout-shoes-HfHgmZ/DV3950-003',
 'https://www.nike.com/in/t/free-metcon-5-workout-shoes-HfHgmZ/DV3950-106',
 'https://www.nike.com/in/t/free-metcon-5-workout-shoes-HfHgmZ/DV3950-0

In [135]:
name2 = []
category2=[]
colors2 = []
price2=[]
available_sizes2 = []
total_sizes_list2 = []
color_name2 = []
product_code2 =[]
review2 =[]
ratings2 =[]
features2 =[]
comfort2 =[]


for i in links1:
    driver.get(i)
    
    #Extracting name====================================
    try:
        nam =driver.find_element(By.XPATH,'//div[@class="prl0-sm mb8-sm"]/div/div/div/div/h1')
        name2.append(nam.text)
        
    except NoSuchElementException:
        name2.append('NA')
        
        
    #Extracting Category============================================================================
    try:
        cate= driver.find_element(By.XPATH,'//div[@class="prl0-sm mb8-sm"]/div/div/div/div/h2')
        category2.append(cate.text)
        
    except NoSuchElementException:
        category2.append('NA')
    
    
    # Extracting colors======================================================================================
    try:
        colr = driver.find_element(By.XPATH,'//fieldset[@class="colorway-images ta-sm-c d-lg-t"]')
        count= len(colr.find_elements(By.XPATH,'//div[@class="css-7aigzk colorway-container d-sm-ib d-lg-tc pr1-sm pb1-sm"]'))
        colors2.append(count)
        
    except NoSuchElementException:
        colors2.append('NA')
        
    # Extracting Price==========================================================================================
    try:
        price_element = driver.find_element(By.XPATH, '//div[@class="prl0-sm mb8-sm"]/div[1]/div/div[2]/div/div/div[1]/div')
        
        #extracting discounted Price
        try:
            reduced = price_element.find_element(By.XPATH, './/div[@data-test="product-price-reduced"]')
            price2.append(reduced.text.split('₹')[1].strip())
        
        except NoSuchElementException:
            try:
                cost = price_element.find_element(By.XPATH, './/div[@data-test="product-price"]')
                price1.append(cost.text.split('₹')[1].strip())
                
            except NoSuchElementException:
                price2.append('NA')
                
    except NoSuchElementException:
        price2.append('NA')
        
    #Extract Sizes and count _of_sizes==================================================================
    try:
        #Count of sizes
        size_table= driver.find_element(By.XPATH, '//div[@class="mt2-sm css-12whm6j"]')
        size_select = size_table.find_elements(By.XPATH, '//input[@name="skuAndSize" and not(@disabled)]')
        total_sizes_list2.append(len(size_select))

        #Extracting Available sizes
        size_text=[size.get_attribute("value").split(':')[1] for size in size_select]
        available_sizes2.append(size_text)
            
    except NoSuchElementException:
        available_sizes2.append('NA')
        total_sizes_list2.append('NA')
        
    # Extract name of the colors==========================================================================
    try:
        colr = driver.find_element(By.XPATH,'//ul[@class="description-preview__features pt8-sm pb6-sm ncss-list-ul"]/li[1]')
        color_name2.append(colr.text.split(':')[1].strip())
        
    except NoSuchElementException:
        color_name2.append('NA')
        
    # Extract Product code========================================================================================
    try:
        code = driver.find_element(By.XPATH,'//ul[@class="description-preview__features pt8-sm pb6-sm ncss-list-ul"]/li[2]')
        product_code2.append(code.text.split(':')[1].strip())
    
    except NoSuchElementException:
        product_code2.append('NA')
        
    #Extracting Review()================================================================================
    try:
        rev = driver.find_element(By.XPATH,'//details[@data-test="reviewsAccordionClick"]/summary/h3/span')
        review2.append(rev.text.split('(')[1].strip(')'))
    
    except NoSuchElementException:
        review2.append('NA')
        
    #extracting Star rating==================================================================================
    try:
        scroll = driver.find_element(By.XPATH,'//details[@data-test="reviewsAccordionClick"]')
        actions = ActionChains(driver)
        actions.move_to_element(scroll).perform()
        scroll.click()
        
    except NoSuchElementException:
        pass
    
    time.sleep(2)
    
    try:
        star_rat = driver.find_element(By.XPATH,'//div[@class="product-review mb10-sm"]')
        
        try:
            rat1 =star_rat.find_element(By.XPATH,'./p[1]')
            ratings2.append(rat1.text.split(' ')[0])
        
        except NoSuchElementException:
            try:
                rat = driver.find_element(By.XPATH,'//div[@class="reviews-component mb5-sm"]/div/p[1]')
                ratings2.append(rat.text.split(' ')[0])
        
            except NoSuchElementException:
                ratings2.append('NA')
                
    except NoSuchElementException:
        ratings2.append('NA')
    
    #Clicking button to extract Product Details======
    
    driver.execute_script("window.scrollBy(0, -1000);")
    
    try:
        button = driver.find_element(By.XPATH,'//button[@class="ncss-cta-primary-dark btn-lg underline prl0-sm pb0-sm readMoreBtn css-6q6seo"]')
        button.click()
        time.sleep(2)
    except NoSuchElementException:
        pass
    
    #extracting Features===========================================================
    try:
        popup= driver.find_element(By.XPATH,'//div[@class="css-1hwqoaj browse-dialog__panel browse-dialog__panel--drop-fade  css-1qiovem"]')
        
        descp =popup.find_element(By.XPATH,'//div[@class="pi-pdpmainbody"]')
        para = descp.find_elements(By.XPATH,'/p')
        features2.append(descp.text)
        
    except NoSuchElementException:
        features1.append('NA')
        
        
    #Extracting Comfort====================================================================
    try:
        comf =popup.find_element(By.XPATH,'//div[@class="pi-pdpmainbody"]/ul[1]')
        comfort2.append(comf.text)
    
    except (NoSuchElementException, StaleElementReferenceException):
        comfort2.append('NA')

In [136]:
len(name2), len(category2), len(colors2), len(price2), len(available_sizes2), len(total_sizes_list2), len(color_name2), len(product_code2),len(review2), len(ratings2), len(features2), len(comfort2)

(500, 500, 500, 227, 500, 500, 500, 500, 500, 500, 499, 500)

In [ ]:
# Converting in DataFrame
Nike_clrurls2nd_1000=pd.DataFrame({
    'Name': name2,
    'Category': category2,
    'Colors': colors1,
    'Price': price1,
    'Sizes': available_sizes1,
    'Count_Sizes': total_sizes_list1,
    'Color_Name': color_name1,
    'product_Code': product_code1,
    'Review': review1,
    'Rating' : ratings1,
    'Features' : features1,
    'Comfort' : comfort1,
    'Product_url': links1
    
})
Nike_clrurls2nd_1000

In [ ]:
# Saving in Csv

Nike_clrurls2nd_1000.to_csv('Nike_clrurls2nd_1000.csv', index=False)

In [34]:
price2 = []
features2 = []

for link in links1:
    driver.get(link)

    # Extracting Price
    try:
        price_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="prl0-sm mb8-sm"]/div[1]/div/div[2]/div/div/div[1]/div'))
        )

        # Extracting discounted Price
        try:
            reduced = price_element.find_element(By.XPATH, './/div[@data-test="product-price-reduced"]')
            price2.append(reduced.text.split('₹')[1].strip())
        except NoSuchElementException:
            cost = price_element.find_element(By.XPATH, './/div[@data-test="product-price"]')
            price2.append(cost.text.split('₹')[1].strip())
    except NoSuchElementException:
        price2.append('NA')

  
    # Scroll directly to the button
    button_xpath = '//button[@class="ncss-cta-primary-dark btn-lg underline prl0-sm pb0-sm readMoreBtn css-6q6seo"]'
    driver.execute_script(f"document.evaluate('{button_xpath}', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue.scrollIntoView();")

    # Wait for the button to be clickable
    try:
        button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
        button.click()
        time.sleep(2)
    except NoSuchElementException:
        pass

    
    #extracting Features===========================================================
    try:
        popup= driver.find_element(By.XPATH,'//div[@class="css-1hwqoaj browse-dialog__panel browse-dialog__panel--drop-fade  css-1qiovem"]')
        
        descp =popup.find_element(By.XPATH,'//div[@class="pi-pdpmainbody"]')
        para = descp.find_elements(By.XPATH,'/p')
        features2.append(descp.text)
        
    except NoSuchElementException:
        features2.append('NA')
        

In [35]:
len(price2),len(features2)

(500, 500)

In [49]:
price2

['7 495.00',
 '9 695.00',
 '9 695.00',
 '9 207.00',
 '9 207.00',
 '9 695.00',
 '19 295.00',
 '19 295.00',
 '19 295.00',
 '10 257.00',
 '10 795.00',
 '10 257.00',
 '10 795.00',
 '10 795.00',
 '10 257.00',
 '10 257.00',
 '10 257.00',
 '4 995.00',
 '3 497.00',
 '4 995.00',
 '4 747.00',
 '2 847.00',
 '2 995.00',
 '2 847.00',
 '2 847.00',
 '2 847.00',
 '2 847.00',
 '2 995.00',
 '2 847.00',
 '12 795.00',
 '12 795.00',
 '12 795.00',
 '12 795.00',
 '7 095.00',
 '7 095.00',
 '7 095.00',
 '8 695.00',
 '8 695.00',
 '5 695.00',
 '5 695.00',
 '5 695.00',
 '9 207.00',
 '9 695.00',
 '9 695.00',
 '3 695.00',
 '3 695.00',
 '3 995.00',
 '3 995.00',
 '3 995.00',
 '3 995.00',
 '3 797.00',
 '3 797.00',
 '8 695.00',
 '8 167.00',
 '8 167.00',
 '8 595.00',
 '17 495.00',
 '17 295.00',
 '17 295.00',
 '7 995.00',
 '7 597.00',
 '7 995.00',
 '11 297.00',
 '11 297.00',
 '11 895.00',
 '11 297.00',
 '11 297.00',
 '11 297.00',
 '11 895.00',
 '11 895.00',
 '11 895.00',
 '11 297.00',
 '14 247.00',
 '10 497.00',
 '7 495.

In [44]:
df.to_csv('Nike_clrurls2nd_1000part2.to_csv', index=False)

In [37]:
df =pd.DataFrame({'Price': price2, 'Features' : features2})
df

,Price,Features
0,7 495.00,INSPIRED BY AN ICON.\n\nA familiar favourite i...
1,9 695.00,"\nInspired by early-2000s running, the Tech H..."
2,9 695.00,"\nInspired by early-2000s running, the Tech H..."
3,9 207.00,"\nInspired by early-2000s running, the Tech H..."
4,9 207.00,"\nInspired by early-2000s running, the Tech H..."
...,...,...
495,11 297.00,\nHoof it from the bottom of the trail to the...
496,8 117.00,ENERGISED FROM ROAD TO TRAIL.\n\nRunning is yo...
497,11 595.00,ENERGISED FROM ROAD TO TRAIL.\n\nRunning is yo...
498,11 595.00,ENERGISED FROM ROAD TO TRAIL.\n\nRunning is yo...


In [4]:
df=pd.read_csv('colr_nike.csv')
df

,color_links
0,https://www.nike.com/in/t/air-force-1-07-shoes...
1,https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
2,https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
3,https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
4,https://www.nike.com/in/t/air-max-1-shoes-ZCSX...
...,...
1601,https://www.nike.com/in/t/phantom-gx-club-mult...
1602,https://www.nike.com/in/t/tiempo-legend-10-clu...
1603,https://www.nike.com/in/t/jordan-why-not-6-pf-...
1604,https://www.nike.com/in/t/jordan-why-not-6-pf-...


In [5]:
color_links = list(df['color_links'])
len(color_links)

1606

In [7]:
links2 = color_links[1000:]
links2

['https://www.nike.com/in/t/pegasus-trail-4-trail-running-shoes-LP9MFR/DJ6158-700',
 'https://www.nike.com/in/t/waffle-debut-vintage-shoes-t44ShF/DX2931-001',
 'https://www.nike.com/in/t/nikecourt-air-zoom-vapor-pro-2-hard-court-tennis-shoes-8cpW40/DR6191-102',
 'https://www.nike.com/in/t/nikecourt-air-zoom-vapor-pro-2-hard-court-tennis-shoes-8cpW40/DR6191-001',
 'https://www.nike.com/in/t/nikecourt-air-zoom-vapor-pro-2-hard-court-tennis-shoes-8cpW40/DR6191-800',
 'https://www.nike.com/in/t/nikecourt-air-zoom-vapor-pro-2-hard-court-tennis-shoes-8cpW40/DR6191-004',
 'https://www.nike.com/in/t/kiger-9-trail-running-shoes-dvCd2S/DR2694-001',
 'https://www.nike.com/in/t/kiger-9-trail-running-shoes-dvCd2S/DR2694-600',
 'https://www.nike.com/in/t/air-jordan-1-mid-se-shoes-HhrDTz/FB9892-002',
 'https://www.nike.com/in/t/pegasus-trail-4-gore-tex-waterproof-trail-running-shoes-pQ2Pzs/DJ7926-003',
 'https://www.nike.com/in/t/pegasus-trail-4-gore-tex-waterproof-trail-running-shoes-pQ2Pzs/DJ7926-0

In [8]:
len(links2)

606

In [9]:
name3 = []
category3=[]
colors3 = []
price3=[]
available_sizes3 = []
total_sizes_list3 = []
color_name3 = []
product_code3 =[]
review3 =[]
ratings3 =[]
features3 =[]
comfort3 =[]


for i in links2:
    driver.get(i)
    
    #Extracting name====================================
    try:
        nam =driver.find_element(By.XPATH,'//div[@class="prl0-sm mb8-sm"]/div/div/div/div/h1')
        name3.append(nam.text)
        
    except NoSuchElementException:
        name3.append('NA')
        
        
    #Extracting Category============================================================================
    try:
        cate= driver.find_element(By.XPATH,'//div[@class="prl0-sm mb8-sm"]/div/div/div/div/h2')
        category3.append(cate.text)
        
    except NoSuchElementException:
        category3.append('NA')
    
    
    # Extracting colors======================================================================================
    try:
        colr = driver.find_element(By.XPATH,'//fieldset[@class="colorway-images ta-sm-c d-lg-t"]')
        count= len(colr.find_elements(By.XPATH,'//div[@class="css-7aigzk colorway-container d-sm-ib d-lg-tc pr1-sm pb1-sm"]'))
        colors3.append(count)
        
    except NoSuchElementException:
        colors3.append('NA')
        
    # Extracting Price==========================================================================================
    try:
        price_element = driver.find_element(By.XPATH, '//div[@class="prl0-sm mb8-sm"]/div[1]/div/div[2]/div/div/div[1]/div')
        
        #extracting discounted Price
        try:
            reduced = price_element.find_element(By.XPATH, './/div[@data-test="product-price-reduced"]')
            price3.append(reduced.text.split('₹')[1].strip())
        
        except NoSuchElementException:
            try:
                cost = price_element.find_element(By.XPATH, './/div[@data-test="product-price"]')
                price3.append(cost.text.split('₹')[1].strip())
                
            except NoSuchElementException:
                price3.append('NA')
                
    except NoSuchElementException:
        price3.append('NA')
        
    #Extract Sizes and count _of_sizes==================================================================
    try:
        #Count of sizes
        size_table= driver.find_element(By.XPATH, '//div[@class="mt2-sm css-12whm6j"]')
        size_select = size_table.find_elements(By.XPATH, '//input[@name="skuAndSize" and not(@disabled)]')
        total_sizes_list3.append(len(size_select))

        #Extracting Available sizes
        size_text=[size.get_attribute("value").split(':')[1] for size in size_select]
        available_sizes3.append(size_text)
            
    except NoSuchElementException:
        available_sizes3.append('NA')
        total_sizes_list3.append('NA')
        
    # Extract name of the colors==========================================================================
    try:
        colr = driver.find_element(By.XPATH,'//ul[@class="description-preview__features pt8-sm pb6-sm ncss-list-ul"]/li[1]')
        color_name3.append(colr.text.split(':')[1].strip())
        
    except NoSuchElementException:
        color_name3.append('NA')
        
    # Extract Product code========================================================================================
    try:
        code = driver.find_element(By.XPATH,'//ul[@class="description-preview__features pt8-sm pb6-sm ncss-list-ul"]/li[2]')
        product_code3.append(code.text.split(':')[1].strip())
    
    except NoSuchElementException:
        product_code3.append('NA')
        
    #Extracting Review()================================================================================
    try:
        rev = driver.find_element(By.XPATH,'//details[@data-test="reviewsAccordionClick"]/summary/h3/span')
        review3.append(rev.text.split('(')[1].strip(')'))
    
    except NoSuchElementException:
        review3.append('NA')
        
    #extracting Star rating==================================================================================
    try:
        scroll = driver.find_element(By.XPATH,'//details[@data-test="reviewsAccordionClick"]')
        actions = ActionChains(driver)
        actions.move_to_element(scroll).perform()
        scroll.click()
        
    except NoSuchElementException:
        pass
    
    time.sleep(2)
    
    try:
        star_rat = driver.find_element(By.XPATH,'//div[@class="product-review mb10-sm"]')
        
        try:
            rat1 =star_rat.find_element(By.XPATH,'./p[1]')
            ratings3.append(rat1.text.split(' ')[0])
        
        except NoSuchElementException:
            try:
                rat = driver.find_element(By.XPATH,'//div[@class="reviews-component mb5-sm"]/div/p[1]')
                ratings3.append(rat.text.split(' ')[0])
        
            except NoSuchElementException:
                ratings3.append('NA')
                
    except NoSuchElementException:
        ratings3.append('NA')
    
    #Clicking button to extract Product Details======
    
    driver.execute_script("window.scrollBy(0, -1000);")
    
    try:
        button = driver.find_element(By.XPATH,'//button[@class="ncss-cta-primary-dark btn-lg underline prl0-sm pb0-sm readMoreBtn css-6q6seo"]')
        button.click()
        time.sleep(2)
    except NoSuchElementException:
        pass
    
    #extracting Features===========================================================
    try:
        popup= driver.find_element(By.XPATH,'//div[@class="css-1hwqoaj browse-dialog__panel browse-dialog__panel--drop-fade  css-1qiovem"]')
        
        descp =popup.find_element(By.XPATH,'//div[@class="pi-pdpmainbody"]')
        para = descp.find_elements(By.XPATH,'/p')
        features3.append(descp.text)
        
    except NoSuchElementException:
        features3.append('NA')
        
        
    #Extracting Comfort====================================================================
    try:
        comf =popup.find_element(By.XPATH,'//div[@class="pi-pdpmainbody"]/ul[1]')
        comfort3.append(comf.text)
    
    except (NoSuchElementException, StaleElementReferenceException):
        comfort3.append('NA')

In [11]:
len(name3), len(category3), len(colors3), len(price3), len(available_sizes3), len(total_sizes_list3), len(color_name3), len(product_code3),len(review3), len(ratings3), len(features3), len(comfort3)

(606, 606, 606, 606, 606, 606, 606, 606, 606, 606, 606, 606)

In [13]:
Nike_clrurls3rd_1600=pd.DataFrame({
    'Name': name3,
    'Category': category3,
    'Colors': colors3,
    'Price': price3,
    'Sizes': available_sizes3,
    'Count_Sizes': total_sizes_list3,
    'Color_Name': color_name3,
    'product_Code': product_code3,
    'Review': review3,
    'Rating' : ratings3,
    'Features' : features3,
    'Comfort' : comfort3,
    'Product_url': links2
    
})
Nike_clrurls3rd_1600

,Name,Category,Colors,Price,Sizes,Count_Sizes,Color_Name,product_Code,Review,Rating,Features,Comfort,Product_url
0,Nike Pegasus Trail 4,Men's Trail-Running Shoes,6,11 017.00,[8],1,Team Gold/Baltic Blue/Stadium Green/Volt,DJ6158-700,200,4.6,ENERGISED FROM ROAD TO TRAIL.\n\nRunning is yo...,Weight: 289g (men's size 9)\nHeel-to-toe drop:...,https://www.nike.com/in/t/pegasus-trail-4-trai...
1,Nike Waffle Debut Vintage,Women's Shoes,2,4 967.00,NA,NA,Light Bone/Team Gold/Gum Medium Brown/Sail,DX2931-001,24,4.4,\nWe've got a secret: these retro kicks are b...,Swoosh wraps around the back to form a pull ta...,https://www.nike.com/in/t/waffle-debut-vintage...
2,NikeCourt Air Zoom Vapor Pro 2,Men's Hard Court Tennis Shoes,5,10 257.00,"[8, 11]",2,White/Kelly Green,DR6191-102,9,4.8,\nFor the rally that won't relent or the mara...,Stretchy internal sleeve hugs your foot for a ...,https://www.nike.com/in/t/nikecourt-air-zoom-v...
3,NikeCourt Air Zoom Vapor Pro 2,Men's Hard Court Tennis Shoes,5,10 257.00,"[7, 9, 12, 12.5]",4,Black/White,DR6191-001,9,4.8,\nFor the rally that won't relent or the mara...,Stretchy internal sleeve hugs your foot for a ...,https://www.nike.com/in/t/nikecourt-air-zoom-v...
4,NikeCourt Air Zoom Vapor Pro 2,Men's Hard Court Tennis Shoes,5,10 795.00,"[7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 1...",13,Ember Glow/White/Noble Red,DR6191-800,9,4.8,\nFor the rally that won't relent or the mara...,Stretchy internal sleeve hugs your foot for a ...,https://www.nike.com/in/t/nikecourt-air-zoom-v...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,Nike Phantom GX Club,Multi-Ground Low-Top Football Boot,4,5 495.00,NA,NA,Black/Dark Smoke Grey/Total Orange/Summit White,DD9483-010,5,5,\nWhether you're starting out or just playing...,For use on natural and synthetic surfaces\nCus...,https://www.nike.com/in/t/phantom-gx-club-mult...
602,Nike Tiempo Legend 10 Club,Multi-Ground Low-Top Football Boot,2,4 995.00,NA,NA,Black/Hyper Royal/Chrome,DV4344-040,7,4.6,\nEven Legends find ways to evolve. Whether y...,For use on natural and synthetic surfaces\nCus...,https://www.nike.com/in/t/tiempo-legend-10-clu...
603,Jordan Why Not .6 PF,Men's Shoes,3,13 297.00,NA,NA,Bright Concord/Black/Vivid Sulphur/Pink Prime,DO7190-460,1,5,\nRussell Westbrook's 6th signature shoe is—y...,PF edition is designed to fit wide for play on...,https://www.nike.com/in/t/jordan-why-not-6-pf-...
604,Jordan Why Not .6 PF,Men's Shoes,3,13 297.00,NA,NA,Bright Crimson/Volt/White/Black,DO7190-607,1,5,\nRussell Westbrook's 6th signature shoe is—y...,PF edition is designed to fit wide for play on...,https://www.nike.com/in/t/jordan-why-not-6-pf-...


In [14]:
# Saving in Csv

Nike_clrurls3rd_1600.to_csv('Nike_clrurls3rd_1600.csv', index=False)